<a href="https://colab.research.google.com/github/ihshin227/tpu/blob/master/keras_attention_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import os
import unicodedata
import re
import io
import time
from sklearn.model_selection import train_test_split

In [0]:
def build_model(input_layer, start_neurons):
  lstm = tf.keras.layers.LSTM(start_neurons, return_sequences=True)(input_layer)
  attention = tf.keras.layers.Attention()([lstm, lstm])
  output_layer = tf.keras.layers.Dense(units=1)(attention)

  return output_layer

In [0]:
inputs = np.random.random([32, 18, 18]).astype(np.float32)
targets = np.random.rand(32).astype(np.float32)

input_layer = tf.keras.layers.Input((18, 18), batch_size=32)
output_layer = build_model(input_layer=input_layer, start_neurons=4)

In [0]:
model = tf.keras.models.Model(input_layer, output_layer)

In [0]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [0]:
model.fit(inputs, targets, epochs=20)

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.3605
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 0.3355
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 0.3130
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2927
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2743
Epoch 6/20
1/1 [==============================] - 0s 866us/step - loss: 0.2578
Epoch 7/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2428
Epoch 8/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2293
Epoch 9/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2171
Epoch 10/20
1/1 [==============================] - 0s 925us/step - loss: 0.2061
Epoch 11/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1962
Epoch 12/20
1/1 [==============================] - 0s 2ms/step - loss: 0.1872
Epoch 13/20
1/1 [==============================] - 0s 1ms/step - loss

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)


In [0]:
interpreter.invoke()

RuntimeError: ignored

In [0]:
tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))

In [0]:
a = model(input_data)
print(a.shape)

(32, 18, 1)


In [0]:
b = model(inputs)
print(b.shape)

(32, 18, 1)


In [0]:
print(input_details[0]['index'])

0


In [0]:
print(input_details[0]['shape'])

[32 18 18]


In [0]:
print(input_data.shape)

(32, 18, 18)
